In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
    !pip install -q transformers
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append('/content/drive/MyDrive/gan_bert_for_cpv')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.9 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
from transformers import logging
logging.set_verbosity_error()
import torch
from create_dataset import CreateDataset
from CPVDataset import CPVDataset
from process_data import ProcessData
from utils import *
from torch.utils.data import DataLoader
from run_model import RunModel
from models import BertGenerator, BertDiscriminator,EmbeddingClassifier
from transformers import AutoTokenizer, AutoModel, AutoConfig

In [3]:
label_column = "afdeling"
text_column = "Korte beschrijving aanbesteding"
input_df = pd.read_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/cleaned_dutch_ted1.csv")
input_df2=pd.read_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/cleaned_foreign_ted1.csv")
all_input=pd.concat([input_df,input_df2])

In [4]:
vc=all_input[label_column].value_counts()

In [5]:
running_device = torch.device("cuda")

In [6]:
prd = ProcessData(all_input, label_column, text_column, sample_num=2000, aug_limit=1000,test_per_label_num=30, drop_limit=50,
                  balanced_test=False)
test_df, suf_df, insuf_df, label_encoder, onehot = prd.process()
balanced_test=test_df.sample(frac=1).groupby(label_column, sort=False).head(30)
value_counts = insuf_df[label_column].value_counts()
model_runner = RunModel(running_device)

After removing duplicate sentence: 263700
After drop cpv with low observations we have : 263700
The num of class is  45
The number of total data is:  76626
Total number of test data is:  15326
Total number of train data is:  58850


In [7]:
value_counts

Installatiediensten (uitgezonderd software)                                                          971
Muziekinstrumenten, sportartikelen, spelletjes, speelgoed, handwerk, kunstartikelen en toebehoren    801
Aardolieproducten, brandstof, elektriciteit en andere energiebronnen                                 785
Machines voor de mijnbouw, steengroeven en voor de bouw                                              724
Mijnbouw, basismetalen en aanverwante producten                                                      686
Landbouwmachines                                                                                     568
Leder, textielweefsels, kunststof- en rubber materialen                                              312
Diensten in verband met de olie- en gasindustrie                                                     259
Landbouw- en veeteelt-, kwekerij-, visserij-, bosbouw- en aanverwante producten                      175
Verzameld en gezuiverd water                           

In [ ]:
from joblib import dump, load
dump(onehot, '/content/drive/MyDrive/gan_bert_for_cpv/output/department/onehot.joblib')

['/content/drive/MyDrive/gan_bert_for_cpv/output/department/onehot.joblib']

In [ ]:
dump(label_encoder, '/content/drive/MyDrive/gan_bert_for_cpv/output/department/labelencoder.joblib')

['/content/drive/MyDrive/gan_bert_for_cpv/output/department/labelencoder.joblib']

In [8]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
config = AutoConfig.from_pretrained("GroNLP/bert-base-dutch-cased", output_hidden_states=True)
bert = AutoModel.from_pretrained("GroNLP/bert-base-dutch-cased", config=config).to(running_device)

In [9]:
cd = CreateDataset(text_column, label_column, label_encoder, onehot, 0, running_device, max_length=100)
suf_dataset=cd.create(suf_df)
test_dataset=cd.create(balanced_test)
insuf_ds_dict=cd.create_label_dataset(insuf_df)

Init new bert


Getting bert hidden states from layer: 0: 100%|██████████| 724/724 [00:09<00:00, 77.78it/s]


In [10]:
insuf_ds_list = list(insuf_ds_dict.values())

In [11]:
test_dl = DataLoader(test_dataset, shuffle=True, batch_size=20)
original_train_dataset = torch.utils.data.ConcatDataset(insuf_ds_list+[suf_dataset])
original_dl = DataLoader(original_train_dataset, shuffle=True, batch_size=20)

In [ ]:
aug_limits = decide_aug_labels(True, 1500, value_counts,1)

In [ ]:
augmented_ds_lists = model_runner.augment_data(bert, aug_limits, insuf_ds_dict,batch_size=20, g_epochs=20, glr=2e-6, dlr=2e-6)

We have: 10 labels to be augmented


100%|██████████| 10/10 [07:43<00:00, 46.36s/it]


In [ ]:
gan_aug_train = torch.utils.data.ConcatDataset([original_train_dataset]+augmented_ds_lists)
gan_aug_dl = DataLoader(gan_aug_train, shuffle=True, batch_size=20)

In [ ]:
classifier = EmbeddingClassifier(prd.num_class, bert)
classifier = model_runner.train_classifier(gan_aug_dl, test_dl, classifier, lr=2e-5, n_epochs=3)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recall, label_f1 = evaluate(labels, predictions)

100%|██████████| 3415/3415 [18:56<00:00,  3.00it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total ACC: 0.516
F1: 0.479
Precision: 0.485
Recall: 0.513
loss 2.2946172751235405 0 acc:  0.5164179104477612 f1:  0.47866513159523805 precision 0.48491051934416723 recall 0.5125925925925926


100%|██████████| 3415/3415 [18:57<00:00,  3.00it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total ACC: 0.575
F1: 0.547
Precision: 0.552
Recall: 0.571
loss 1.490921624897165 1 acc:  0.5753731343283582 f1:  0.5474035539794648 precision 0.5518367568901287 recall 0.5711111111111111


100%|██████████| 3415/3415 [18:58<00:00,  3.00it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total ACC: 0.583
F1: 0.560
Precision: 0.570
Recall: 0.579
loss 1.1394747495302169 2 acc:  0.5828358208955224 f1:  0.5604615804840217 precision 0.57044290705204 recall 0.5785185185185184
Total ACC: 0.583
F1: 0.560
Precision: 0.570
Recall: 0.579


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
classifier = model_runner.train_classifier(gan_aug_dl, test_dl, classifier, lr=2e-5, n_epochs=3)

100%|██████████| 3415/3415 [18:57<00:00,  3.00it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total ACC: 0.594
F1: 0.584
Precision: 0.602
Recall: 0.590
loss 0.869237371597379 0 acc:  0.5940298507462687 f1:  0.5839029007320344 precision 0.6016221051764714 recall 0.5896296296296296


100%|██████████| 3415/3415 [18:56<00:00,  3.00it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total ACC: 0.591
F1: 0.574
Precision: 0.586
Recall: 0.587
loss 0.6758584772123468 1 acc:  0.591044776119403 f1:  0.5744034504600826 precision 0.5857283231428408 recall 0.5866666666666667


100%|██████████| 3415/3415 [18:58<00:00,  3.00it/s]


Total ACC: 0.601
F1: 0.584
Precision: 0.595
Recall: 0.596
loss 0.5511524523751562 2 acc:  0.6007462686567164 f1:  0.584467718555278 precision 0.5950215278434363 recall 0.5962962962962963


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
torch.save(classifier,'/content/drive/MyDrive/gan_bert_for_cpv/output/department/model')

In [ ]:
del test_dataset,suf_dataset

In [ ]:
all_test_ds=cd.create(test_df)

Getting bert hidden states from layer: 0: 100%|██████████| 15326/15326 [03:16<00:00, 77.80it/s]


In [ ]:
all_test_dl = DataLoader(all_test_ds, shuffle=True, batch_size=20)
predictions, labels = model_runner.predict(all_test_dl, classifier)
acc, f1, precision, recall, label_f1 = evaluate(labels, predictions)

Total ACC: 0.614
F1: 0.594
Precision: 0.598
Recall: 0.599


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
classifier = EmbeddingClassifier(prd.num_class, bert)
classifier = model_runner.train_classifier(original_dl, test_dl, classifier, lr=2e-5, n_epochs=3)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recall, label_f1 = evaluate(labels, predictions)

100%|██████████| 2943/2943 [18:46<00:00,  2.61it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total ACC: 0.483
F1: 0.411
Precision: 0.394
Recall: 0.479
loss 2.6842298659588097 0 acc:  0.4828358208955224 f1:  0.4114007546054209 precision 0.39446853712732305 recall 0.47925925925925916


100%|██████████| 2943/2943 [18:46<00:00,  2.61it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total ACC: 0.573
F1: 0.528
Precision: 0.526
Recall: 0.569
loss 1.7155984601438188 1 acc:  0.573134328358209 f1:  0.5281322750777392 precision 0.5256176249799389 recall 0.5688888888888889


100%|██████████| 2943/2943 [18:45<00:00,  2.62it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total ACC: 0.593
F1: 0.555
Precision: 0.547
Recall: 0.589
loss 1.3415640259919182 2 acc:  0.5932835820895522 f1:  0.5547859097861386 precision 0.5470145474424896 recall 0.5888888888888889
Total ACC: 0.593
F1: 0.555
Precision: 0.547
Recall: 0.589


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
classifier = model_runner.train_classifier(original_dl, test_dl, classifier, lr=2e-5, n_epochs=3)

100%|██████████| 2943/2943 [18:34<00:00,  2.64it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total ACC: 0.610
F1: 0.575
Precision: 0.564
Recall: 0.606
loss 1.0357645988980808 0 acc:  0.6104477611940299 f1:  0.5752225365468052 precision 0.5640777761790144 recall 0.6059259259259259


100%|██████████| 2943/2943 [18:46<00:00,  2.61it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total ACC: 0.606
F1: 0.569
Precision: 0.556
Recall: 0.601
loss 0.8330304884825521 1 acc:  0.6059701492537314 f1:  0.5690796951403702 precision 0.5563998197756205 recall 0.6014814814814814


100%|██████████| 2943/2943 [18:48<00:00,  2.61it/s]


Total ACC: 0.610
F1: 0.582
Precision: 0.578
Recall: 0.606
loss 0.6863450684391325 2 acc:  0.6104477611940299 f1:  0.5818304015549954 precision 0.5776366371890657 recall 0.605925925925926


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
